In [ ]:
!wget -O model_repository/densenet_onnx/1/model.onnx https://github.com/onnx/models/raw/main/validated/vision/classification/densenet-121/model/densenet-7.onnx


In [ ]:
!docker run --rm -p8000:8000 -p8001:8001 -p8002:8002 -v/home/trung/PycharmProjects/ojt/host_model/model_repository:/models nvcr.io/nvidia/tritonserver:23.04-py3 tritonserver --model-repository=/models


In [1]:
import numpy as np
import tritonclient.http as httpclient
from PIL import Image
from torchvision import transforms
from tritonclient.utils import triton_to_np_dtype

In [11]:
def rn50_preprocess(img_path="img1.jpg"):
    img = Image.open(img_path)
    preprocess = transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    return preprocess(img).numpy()

In [12]:
transformed_img = rn50_preprocess()

# Setting up client
client = httpclient.InferenceServerClient(url="localhost:8000")

inputs = httpclient.InferInput("data_0", transformed_img.shape, datatype="FP32")
inputs.set_data_from_numpy(transformed_img, binary_data=True)

outputs = httpclient.InferRequestedOutput("fc6_1", binary_data=True, class_count=1000)

# Querying the server
results = client.infer(model_name="densenet_onnx", inputs=[inputs], outputs=[outputs])
inference_output = results.as_numpy("fc6_1").astype(str)

print(np.squeeze(inference_output)[:5])

['11.547369:92:BEE EATER' '11.230166:14:INDIGO FINCH'
 '7.527154:95:JACAMAR' '6.921648:17:JAY' '6.576403:88:MACAW']


In [5]:
pwd

'/home/trung/PycharmProjects/ojt/host_model'